In [ ]:
!pip install Sastrawi
!pip install rapidfuzz

from rapidfuzz import fuzz
from rapidfuzz import process

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

from nltk.tokenize import word_tokenize
from pathlib import Path
from zipfile import ZipFile

from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from sklearn.impute import SimpleImputer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tv = TfidfVectorizer(max_features=5000)
stem = StemmerFactory().create_stemmer()
stopword = StopWordRemoverFactory().create_stop_word_remover()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 29.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Membaca dataset tourism
tourism = pd.read_csv('/content/drive/MyDrive/datasetRevisi.csv', encoding='latin-1')

# Membaca dataset rating
rating = pd.read_csv('/content/drive/MyDrive/ratingDataset.csv')

In [ ]:
#Join/merge dataset tourism dan rating berdasarkan id tempat wisata(place_id)
df = tourism.merge(rating, how="left", on="Place_Id")
#menampilkan beberapa data pada dataframe
df.head()

In [ ]:
def preprocessing(data):
    data = data.lower()
    data = stem.stem(data)
    data = stopword.remove(data)
    return data

In [ ]:
data_content_based_filtering = tourism.copy()
data_content_based_filtering['Tags']  = data_content_based_filtering['Description'] + ' ' + data_content_based_filtering['Category'] + data_content_based_filtering['City']
data_content_based_filtering.drop(['Weekend_Price','Weekday_Price','Gambar','Coordinate','Lat', 'Long','Unnamed: 13',],axis=1,inplace=True)
data_content_based_filtering

In [ ]:
data_content_based_filtering.Tags = data_content_based_filtering.Tags.apply(preprocessing)
# data_content_based_filtering.Place_Name = data_content_based_filtering.Place_Name.apply(preprocessing)
data_content_based_filtering

,Place_Id,Place_Name,Description,Category,City,Rating,Alamat,Tags
0,2,Agung Bali,Dapatkan berbagai produk oleh-oleh khas Bali b...,Belanja,Badung,4.0,"Jln. Dewi Sri No.18XX, Kuta, Badung, Bali",bagai produk khas bal upa baju vip camil handy...
1,10,Bali Swing,Bali Swing Tegalalang yang menggantung di anta...,Rekreasi,Badung,4.5,"Jl. Dewi Saraswati No.7, Bongkasa Pertiwi, Kec...",bal swing tegalalang gantung antara pohon pale...
2,22,Garuda Wisnu Kencana,Patung Garuda Wisnu Kencana menjadi ikon pariw...,Budaya,Badung,3.5,"Jl. Raya Uluwatu, Ungasan, Kec. Kuta Sel., Kab...",patung garuda wisnu kencana jadi ikon pariwisa...
3,29,Pantai Jimbaran,Pantai Jimbaran Bali adalah salah satu pantai ...,Pantai,Badung,4.0,"Kecamatan Kuta selatan, kabupaten Badung, Prov...",pantai jimbaran bal salah satu pantai pasir pu...
4,30,Pabrik Kata-Kata Joger,Bali Joger terkenal di kalangan wisatawan seba...,Belanja,Badung,4.0,"Jl. Raya Kuta, Kuta, Kec. Kuta, Kabupaten Badu...",bal joger kenal kalang wisatawan bagai tempat ...
...,...,...,...,...,...,...,...,...
70,50,Pantai Pasih Uug,Broken Beach atau yang lebih dikenal sebagai P...,Pantai,Klungkung,4.5,"Lembongan, Kec. Nusa Penida, Kabupaten Klungku...",broken beach lebih kenal bagai pasih uug warga...
71,16,Danau Beratan Bedugul,Danau Beratan Bedugul merupakan sebuah danau y...,Alam,Tabanan,4.5,"Kec. Baturiti, Kabupaten Tabanan, Bali",danau rat bedugul rupa buah danau bal keliling...
72,28,Jatiluwih Green Land,Nominasi untuk penunjukan sebagai situs Warisa...,Agrowisata,Tabanan,4.5,"Jl. Jatiluwih Kawan No.Desa, Jatiluwih, Kec. P...",nominasi tunjuk bagai situs waris dunia unesco...
73,61,Pura Luhur Batukaru,Pura Luhur Batukaru adalah sebuah pura yang te...,Religius,Tabanan,4.5,"Jl. Penatahan - Wongayagede, Wongaya Gede, Kec...",pura luhur batukaru buah pura letak lereng gun...


In [ ]:
# content based filtering
def recommend_by_content_based_filtering(nama_tempat):
    matched_tempats = []

    for index, tempat in data_content_based_filtering.iterrows():
        nama_tempat_similarity = fuzz.token_set_ratio(nama_tempat.lower(), tempat['Place_Name'].lower())
        city_similarity = fuzz.token_set_ratio(nama_tempat.lower(), tempat['City'].lower())
        combined_similarity = (nama_tempat_similarity + city_similarity) / 2
        matched_tempats.append((index, combined_similarity))

    matched_tempats = sorted(matched_tempats, key=lambda x: x[1], reverse=True)
    matched_tempats = matched_tempats[:10]  # Ambil 10 tempat terbaik

    recommended_nama_tempats = []
    for match in matched_tempats:
        tempat_index = match[0]
        place_name = data_content_based_filtering.iloc[tempat_index]['Place_Name']
        place_id = data_content_based_filtering.iloc[tempat_index]['Place_Id']
        recommended_nama_tempats.append([place_name, place_id])

    return recommended_nama_tempats




In [ ]:
recommend_by_content_based_filtering('pantai murah di badung')

[['Pantai Kuta', 45],
 ['Pantai Nusa Dua', 47],
 ['Pantai Canggu', 39],
 ['Pantai Padang Padang', 48],
 ['Pantai Pandawa', 49],
 ['Pantai Melasti', 46],
 ['Pantai Jimbaran', 29],
 ['Pantai Seminyak', 54],
 ['Pantai Double Six', 42],
 ['Pantai Dreamland', 43]]